In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,194 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,420 kB]
Hit:13 https://ppa.launchpadcontent.ne

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [9]:
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [7]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')

In [20]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""
SELECT
  YEAR (date) as YEAR,
  ROUND (AVG(price),2) as AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR (date)
ORDER BY YEAR (date)
""").show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [25]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
SELECT
  YEAR (date_built) as YEAR_BUILT,
  ROUND (AVG(price),2) as AVERAGE_PRICE
FROM home_sales
WHERE
bedrooms = 3
and bathrooms = 3
GROUP BY YEAR (date_built)
ORDER BY YEAR (date_built)
""").show()

+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [26]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
SELECT
  YEAR (date_built) as YEAR_BUILT,
  ROUND (AVG(price),2) as AVERAGE_PRICE
FROM home_sales
WHERE
bedrooms = 3
and bathrooms = 3
and floors = 2
and sqft_living >=2000
GROUP BY YEAR (date_built)
ORDER BY YEAR (date_built)
""").show()

+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [29]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT
  view as VIEW_RATING,
  ROUND (AVG(price),2) as AVERAGE_PRICE
FROM home_sales
WHERE
price >= 350000
GROUP BY view
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|VIEW_RATING|AVERAGE_PRICE|
+-----------+-------------+
|          0|    403848.51|
|          1|    401044.25|
|         10|    401868.43|
|        100|    1026669.5|
|         11|    399548.12|
|         12|    401501.32|
|         13|    398917.98|
|         14|    398570.03|
|         15|     404673.3|
|         16|    399586.53|
|         17|    398474.49|
|         18|    399332.91|
|         19|    398953.17|
|          2|    397389.25|
|         20|    399522.81|
|         21|    399758.88|
|         22|    402022.68|
|         23|    403411.92|
|         24|    400284.92|
|         25|    401298.69|
+-----------+-------------+
only showing top 20 rows

--- 1.0974736213684082 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [32]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT
  view as VIEW_RATING,
  ROUND (AVG(price),2) as AVERAGE_PRICE
FROM home_sales
WHERE
price >= 350000
GROUP BY view
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))



+-----------+-------------+
|VIEW_RATING|AVERAGE_PRICE|
+-----------+-------------+
|          0|    403848.51|
|          1|    401044.25|
|         10|    401868.43|
|        100|    1026669.5|
|         11|    399548.12|
|         12|    401501.32|
|         13|    398917.98|
|         14|    398570.03|
|         15|     404673.3|
|         16|    399586.53|
|         17|    398474.49|
|         18|    399332.91|
|         19|    398953.17|
|          2|    397389.25|
|         20|    399522.81|
|         21|    399758.88|
|         22|    402022.68|
|         23|    403411.92|
|         24|    400284.92|
|         25|    401298.69|
+-----------+-------------+
only showing top 20 rows

--- 0.553839921951294 seconds ---


In [34]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("date_built_partitioned")


In [35]:
# 11. Read the parquet formatted data.
db_p=spark.read.parquet('date_built_partitioned')

In [36]:
# 12. Create a temporary table for the parquet data.
db_p.createOrReplaceTempView('date_built_partitioned_table')

In [38]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT
  view as VIEW_RATING,
  ROUND (AVG(price),2) as AVERAGE_PRICE
FROM home_sales
WHERE
price >= 350000
GROUP BY view
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|VIEW_RATING|AVERAGE_PRICE|
+-----------+-------------+
|          0|    403848.51|
|          1|    401044.25|
|         10|    401868.43|
|        100|    1026669.5|
|         11|    399548.12|
|         12|    401501.32|
|         13|    398917.98|
|         14|    398570.03|
|         15|     404673.3|
|         16|    399586.53|
|         17|    398474.49|
|         18|    399332.91|
|         19|    398953.17|
|          2|    397389.25|
|         20|    399522.81|
|         21|    399758.88|
|         22|    402022.68|
|         23|    403411.92|
|         24|    400284.92|
|         25|    401298.69|
+-----------+-------------+
only showing top 20 rows

--- 1.1194698810577393 seconds ---


In [41]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [42]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")


all clear
